import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
for i in range (1,3):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Ecg200(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,3):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
print('shape ecg=',np.shape(ecg))

1 31	2 69	shape ecg= (100, 481)


<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [3]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,3):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,3):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 31	2 69	
 max = 69


In [4]:
#clses=3                                   #تعیین میزان افزایش نمونه (تولید داده)
mx_wndws=int(mx_wndws+6)  #mx_wndws*0.3  # =classes-1 برای بررسی الگوریتم تعداد کمی کلاس آزمایش شد
print(mx_wndws)

75


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [5]:
methd=70         #10=scle,20MgWrp,30=timWrp,40=GDO,50=frqnc,60=GAN,70=LSTMplus
sort=-3          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
dp_slct_dstrb=1  #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm

for i in range (1,3):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+str(methd)]=np.empty((0,smpl_rte+1), float) 
for i in range (1,3):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    print("vlum_win=",vlum_win)
    vars()['ecg_aug'+str(i)+str(methd)]=np.append(vars()['ecg_aug'+str(i)+str(methd)],srtd_data(methd,i,vlum_win,dp_slct_dstrb,sort),axis=0)
    #print('aug shape=',np.shape(scl_rndm(ecg,i,vlum_win,mag=.1)))
    print('generated data shape for class ', i, ' = ', np.shape(vars()['ecg_aug'+str(i)+str(methd)]))

vlum_win= 44
final shape=  (44, 487)
generated data shape for class  1  =  (44, 481)
vlum_win= 6
final shape=  (6, 487)
generated data shape for class  2  =  (6, 481)


In [6]:
srtd_data(methd,i,vlum_win,dp_slct_dstrb,sort)

final shape=  (6, 487)


array([[-2.26148748, -2.94986224, -3.32506752, ...,  0.07004287,
        -0.37712514,  2.        ],
       [-2.13354731, -2.82962036, -3.22518563, ...,  0.1860228 ,
        -0.3067877 ,  2.        ],
       [-2.25758791, -2.9403069 , -3.28782749, ...,  0.03543474,
        -0.70586377,  2.        ],
       [-2.13196182, -2.79582834, -3.10941863, ...,  0.01760381,
        -1.1108439 ,  2.        ],
       [-0.29993334, -0.50433391, -0.72216332, ..., -0.00365712,
         0.49011168,  2.        ],
       [-1.88488054, -2.51619744, -2.78975487, ..., -0.03102844,
        -1.26965797,  2.        ]])

In [7]:
vlum_win

6

In [8]:
           #add augmented data to base data
xtrain=np.empty((0,smpl_rte+1), float) 
for i in range (1,3):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    xtrain=np.append(xtrain,vars()['ecg_aug'+str(i)+str(methd)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (75, 481)
Up to class  2 train shape =  (150, 481)


In [9]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 75 75]


In [10]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,3):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [11]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [12]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,3):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [13]:
#print(np.max(wndws_tst))
for i in range (1,3):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,3):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  4.159687838862726
min magnitude class 1  =  -3.0608885355224738
after normalizing >>
max magnitude class 1  =  0.17187054952831882
min magnitude class 1  =  -0.2127769591304448

 cls 2  >> 
max magnitude class 2  =  4.1476
min magnitude class 2  =  -2.9588110682839144
after normalizing >>
max magnitude class 2  =  0.17122661795519534
min magnitude class 2  =  -0.2073391877632177
each class and its windows =
1 36	2 64	
 max instance in classes of test windows = 64


In [14]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.2127769591304448
max tst = 0.17187054952831882


In [15]:
len(wndws_test)

100

In [16]:
np.shape(wndws_test)

(100, 481)

In [17]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [18]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [19]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [20]:
np.shape(wndws1)

(31, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [21]:
Xvalid=np.array(Xtest)
yvalid=np.array(ytest)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [22]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtest)
y_valid=np.array(ytest)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (150, 480)
y_train =>  (150,)
X_test  =>  (100, 480)
y_test  =>  (100,)
X_valid  =>  (100, 480)
y_valid  =>  (100,)


In [23]:
y_train=np.array(ytrain)
np.min(y_train)

1

In [24]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [25]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 5
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
30/30 [==============================] - 9s 16ms/step - loss: 0.7889 - accuracy: 0.4867 - val_loss: 0.6835 - val_accuracy: 0.6400
Epoch 2/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.6974 - accuracy: 0.5600 - val_loss: 0.6452 - val_accuracy: 0.7300
Epoch 3/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.5678 - accuracy: 0.7533 - val_loss: 0.5299 - val_accuracy: 0.7700
Epoch 4/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.4504 - accuracy: 0.7933 - val_loss: 0.5187 - val_accuracy: 0.7600
Epoch 5/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.3966 - accuracy: 0.8400 - val_loss: 0.5018 - val_accuracy: 0.7300
Epoch 6/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.4526 - accuracy: 0.7867 - val_loss: 0.4691 - val_accuracy: 0.7800
Epoch 7/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.3587 - accuracy: 0.8467 - val_loss: 0.4310 - val_accuracy: 0.7600

Epoch 58/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.6465 - val_accuracy: 0.9300
Epoch 59/1000
30/30 [==============================] - 0s 10ms/step - loss: 7.9480e-05 - accuracy: 1.0000 - val_loss: 0.6481 - val_accuracy: 0.9200
Epoch 60/1000
30/30 [==============================] - 0s 10ms/step - loss: 3.0147e-05 - accuracy: 1.0000 - val_loss: 0.6740 - val_accuracy: 0.9200
Epoch 61/1000
30/30 [==============================] - 0s 11ms/step - loss: 1.2390e-05 - accuracy: 1.0000 - val_loss: 0.7048 - val_accuracy: 0.9100
Epoch 62/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.1369 - accuracy: 0.9733 - val_loss: 1.7242 - val_accuracy: 0.8900
Epoch 63/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.0486 - accuracy: 0.9867 - val_loss: 0.8205 - val_accuracy: 0.9200
Epoch 64/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.0718 - accuracy: 0.9800 - val_loss: 0.9239 - v

In [26]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [27]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
print(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

4/4 [==============================] - 0s 20ms/step
[[35  1]
 [ 4 60]]
              precision    recall  f1-score   support

           1       0.90      0.97      0.93        36
           2       0.98      0.94      0.96        64

    accuracy                           0.95       100
   macro avg       0.94      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100



In [28]:
print(np.bincount(y_train)[1:])

[75 75]
